### Introducción a la materia Aprendizaje Automático

**Un juego**

Un amigo entrañable, Alex, se fue a vivir a EEUU hace unos años. Hace un rato mandó mensajito de WhatsApp, después de un par de meses sin tener noticias. Nos contó que había conseguido un trabajo muy bien pago en una inmobiliaria en San Francisco, California.  Claro que para hacerlo, había tenido que poner en su CV que era "Data Scientists". Alex nos explica que es lo que está haciendo todo el mundo, porque piden Data Scientists por doquier. Pero en realidad su formación en estadística y análisis de datos se limita a un par de videos de YouTube, y a un cursito de Coursera, que dejó por la mitad. 

El primer lunes en la oficina, nos cuenta Alex, su jefe llega, exultante, y le cuenta lo satisfecho que está de iniciar su departamento de Data Science en la Inmobiliaria, y le pasa la clave para acceder a los archivos de la empresa, que contiene información de las propiedades que tiene en su haber la inmobiliaria. Y le pide que los use para "mejorar la eficiencia y ganancia de la inmobiliaria". Le pide algún resultado para el final de la semana.

Alex está en pánico, pero vio el tuit que vos había escrito, diciendo que arrancabas la materia de Aprendizaje Automático de UNSAM, y pensó que tal vez podés darle una mano para salir de este brete. Promete conectarse como oyente a las clases de ahora en más. Antes de esperar la respuesta, manda los datos y un gif simpático.

Vamos a ver cómo podemos ayudarlo, pobre Alex.

**Plan**

1) Visualizar los datos para obtener alguna idea de qué pregunta podrían llegar a responder.

2) Definir el proyecto.

3) Preparar los datos.

4) Correr algún modelo simple simple (¿tal vez el jueves?)

## Preparación

Antes que nada, corramos algunas celdas de código para prepararnos. Mucho de lo que viene a continuación está sacado del libro de Aurélien Geron, y su [repo de GitHub](https://github.com/ageron/handson-ml2), que recomendamos.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "01_Intro"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Obtención de los datos

Lo primero es obtener los datos que manda Alex. Para eso, definimos una variable extra, con la ubicación del archivo

In [ ]:
HOUSING_PATH = os.path.join(".", "datasets", "housing")

Si están corriendo esto en Colab, tienen que bajar los datos del repositorio y copiarlo a su directorio en la nube. Esto se hace con el código de la siguiente celda

In [ ]:
assert 'google.colab' in sys.modules, "Ojo! No estás corriendo en Colab. No hace falta esto."
    
import tarfile

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/IAI-UNSAM/ML_UNSAM/master/"
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

!mkdir -p ./datasets/housing

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    #urllib.request.urlretrieve(housing_url, tgz_path)
    !wget https://raw.githubusercontent.com/IAI-UNSAM/ML_UNSAM/master/datasets/housing/housing.tgz -P {housing_path}
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
# Corramos la función
fetch_housing_data()

In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [ ]:
#Cargemos los datos y veamos las primeras filas
housing = load_housing_data()
housing.head()

> Yo: "Alex, ¿qué es esto? Tirame alguna pista adicional."
>
> Alex: "Son datos por distrito. Cada fila representa un distrito diferente en California."
>
> Yo: "Algo más?"
>
> Alex: ...
> 
> Yo: "Las unidades de cada columna?"
>
> Alex: ...

Creo que se dice "nos clavó el visto".

Bueno, por suerte el paquete `pandas` tiene cosas útiles. Usemos el método <tt>info</tt> para tener más información.

In [ ]:
housing.info()

Vemos que tenemos 20640 registros (distritos), con 10 columnas para cada uno.

La mayoría de esas columnas son numéricas, pero hay una que tiene otro tipo de objetos (ya lo vimos arriba).

`ocean_proximity` es intrigante. Veamos más de cerca. ¿Qué valores toma?

In [ ]:
housing["ocean_proximity"].unique()

Ok, vemos que es lo que se llama una variable categórica. Hay solo cinco posibles valores para esa variable. Veamos cuantos distritos en cada uno.

***

**Preguntas**

1) ¿Qué preguntas creen que pueden responder estos datos?

2) ¿Cómo haríamos para empezar a responder esa pregunta?

3) ¿Qué variables parecen las más relevantes?

## Criterio de evaluación.

Una vez que tenemos una idea clara de los objetivos, lo primero que tenemos que hacer es decidir qué criterio de evaluación vamos a usar.

***

**Preguntas**

1) ¿Cómo podemos saber si nuestro modelo funciona?

2) ¿Cómo podemos cuantíficar cuán bien funciona?

3) ¿Cómo garantizamos que la predicción va a funcionar cuando aparezcan nuevas casas para evaluar?

## Exploración de los datos

Tal vez lo primero que a uno se le puede ocurrir es calcular promedios y cosas de los datos.

In [ ]:
housing.describe().round(2)

Vean que `ocean_proximity` no aparece en esta tabla, porque no es numérica. Y volvemos a ver que faltan algunas entrdas en `total_bedrooms`.

***
**Pregunta**: ¿Se les ocurre que podemos hacer con los registros que faltan de `total_bedrooms`?
***
Otra buena idea es hacer un plot de los gráficos. Como tenemos latitud y longitud, se impone graficar eso.

In [ ]:
ax = housing.plot(kind="scatter", x="longitude", y="latitude", figsize=(8, 8))
# save_fig("bad_visualization_plot")

Mmmm, no se ve nada, no? Aunque se adivina la forma de California.

<p><a href="https://commons.wikimedia.org/wiki/File:Map_of_California_NA-2004-compact.png#/media/File:Map_of_California_NA-2004-compact.png"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/78/Map_of_California_NA-2004-compact.png/1200px-Map_of_California_NA-2004-compact.png" alt="Map of California NA-2004-compact.png" style="width:400px; align:left"></a></p>

Podemos agregar algo de claridad haciendo que los puntos sean transparentes.

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1, figsize=(8, 8))
# save_fig("better_visualization_plot")

Ahora podemos ver que hay zonas más pobladas y otras menos pobladas, por ejemplo.

Por último agreguemos otras variables como tamaño de los símbolos y color.

*** 

*Pregunta*

1) ¿Qué variable agregarían?

----

In [ ]:
housing.head()

Decidimos agregar la mediana del valor de las casas y la población. Pueden experimentar con otros, a ver qué pasa.

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", s=housing["population"]/50, label="población", 
             c="median_house_value", figsize=(8, 8), alpha=0.4, cmap=plt.get_cmap("jet"), colorbar=True,
             sharex=False)

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", s=housing["median_income"]*10, label="Ingreso", 
             c="median_house_value", figsize=(8, 8), alpha=0.1, cmap=plt.get_cmap("jet"), colorbar=True,
             sharex=False)

Epa! Bueno, esto ya es otra cosa! Solo con este gráfico tenemos mucha tela para cortar.

Pero si quieren algo todavía más bonito, pueden usar este código (gracias, Geron!)

In [ ]:
# Download the California image
images_path = os.path.join(PROJECT_ROOT_DIR, "images", "01_Intro")
os.makedirs(images_path, exist_ok=True)
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
filename = "california.png"
print("Downloading", filename)
url = DOWNLOAD_ROOT + "images/end_to_end_project/" + filename
!wget {url} -P {images_path}
# !mv california.png {images_path}
# urllib.request.urlretrieve(url, os.path.join(images_path, filename))

In [ ]:
import matplotlib.image as mpimg
california_img=mpimg.imread(os.path.join(images_path, filename))
ax = housing.plot(kind="scatter", x="longitude", y="latitude", figsize=(10,7),
                       s=housing['population']/100, label="Population",
                       c="median_house_value", cmap=plt.get_cmap("jet"),
                       colorbar=False, alpha=0.4,
                      )
plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5,
           cmap=plt.get_cmap("jet"))
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)

prices = housing["median_house_value"]
tick_values = np.linspace(prices.min(), prices.max(), 11)
cbar = plt.colorbar(ticks=tick_values/prices.max())
cbar.ax.set_yticklabels(["$%dk"%(round(v/1000)) for v in tick_values], fontsize=14)
cbar.set_label('Median House Value', fontsize=16)

plt.legend(fontsize=16)
# save_fig("california_housing_prices_plot")
plt.show()

In [ ]:
housing.describe()

In [ ]:
housing['median_house_value'].hist(bins=50)

# a = housing['median_house_value'].to_numpy()
# print(type(housing['median_house_value']))
# print(type(a))

# plt.hist(a, 50)

In [ ]:
sum(housing['median_house_value'] == housing.median_house_value.max())

Hay 965 distritos que fueron asignados al valor más alto (501 000). Es decir, que en el proceso de obtención de datos, se cortó en ese valor el precio de las casas.

### Variable categórica

In [ ]:
housing["ocean_proximity"].unique()

In [ ]:
housing["ocean_proximity"].value_counts()

Un poco más visual.

In [ ]:
ocean_group = housing.groupby('ocean_proximity')
ocean_group.count().plot(kind='bar')

Hago la estadística para cada categoría de una de las variables (en este caso, "median house value")

In [ ]:
ocean_group.describe()['median_house_value'].round(2)

Vemos que el precio medio depende fuertemente de la ubicación con respecto al océano, como ya habíamos visto en el plot.

In [ ]:
corr_matrix = housing.corr()

In [ ]:
housing.plot(kind='scatter', x='total_rooms', y='total_bedrooms')

In [ ]:
plt.pcolor(corr_matrix)
plt.yticks(np.arange(0.5, len(corr_matrix.index), 1), corr_matrix.index, rotation=0)
plt.xticks(np.arange(0.5, len(corr_matrix.columns), 1), corr_matrix.columns, rotation=90)
plt.colorbar()

In [ ]:
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
ax = housing.plot(kind='scatter', x='median_income', y='median_house_value', alpha=0.1)
ax.axvline(housing.median_income.mean(), color='r')
ax.axhline(housing.median_house_value.mean(), color='r')
ax.plot()

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
sm = scatter_matrix(housing[attributes], figsize=(12, 8), hist_kwds={'bins':100})

In [ ]:
scatter_matrix??

In [ ]:
housing.median_income.describe()

In [ ]:
import pandas as pd
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

In [ ]:
income_group = housing.groupby("income_cat")
income_group.describe()['median_house_value'].round(2)

Segementamos por ingreso, pero el número de bins y los límites son arbitrarios.

### "infinitos" bins (regresión lineal)

$$
\hat{\text{median_house_value}} = \omega_0 + \omega_1*\text{median_income}
$$

$$
y = \omega_0 + \omega_1 * x
$$

$\omega_0$ y $\omega_1$ son los parámetros de mi modelo.

In [ ]:
# Variables predictoras
x = housing.median_income.to_numpy()
x = x.reshape([-1, 1])

# Labels (valores reales) / Targets
t = housing.median_house_value.to_numpy()


from sklearn.linear_model import LinearRegression

lr = LinearRegression()

A partir de mi predicción $y$, busco minimizar el MSE (error cuadrático medio)

$$
\frac{1}{N}\sum_{i=1}^{N}(y_i - t_i)^2
$$

Es más piola usar el RMSE (raíz del error cuadrático medio)

$$
\sqrt{\frac{1}{N}\sum_{i=1}^{N}(y_i - t_i)^2}
$$


In [ ]:
lr.fit(x, t)

In [ ]:
y = lr.predict(x)

In [ ]:
plt.hist(y, histtype='step', label='predicción')
plt.hist(t, histtype='step', label='target')
plt.legend(loc=0)

In [ ]:
rmse = np.sqrt(np.sum((y - t)**2)/len(y))
print('El RMSE del modelo lineal es {:.3f}'.format(rmse))

In [ ]:
plt.plot(x, t, '.b', alpha=0.1)
plt.plot(x, y, 'or')
plt.xlabel('x = median_income')
plt.ylabel('y = median_house_value')

In [ ]:
lr.coef_, lr.intercept_